<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/PySyft_Simple_Flower102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports, setup


In [21]:
import time
import datetime
import logging
import math
import os
import sys

from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline


# save the model on Google Drive, link Google drive to this notebook
from google.colab import drive
drive.mount('/content/gdrive')


# After executing this cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/gdrive/My Drive/Colab Notebooks/flower_data/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
picco_test  test  train  train_ofgdrive  valid	valid_ofgdrive


In [0]:
#!rm -rf ./PySyft
  

### Manual installation of PySyft necessary due to a PySyft bug 

In [2]:
!pip install tf-encrypted
! URL="https://github.com/Berenice2018/PySyft-Bc.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b master --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /master/null

module_path = os.path.abspath(os.path.join('./PySyft-Bc'))
if module_path not in sys.path:
     sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
Successfully built zstd


In [0]:
#!cat './PySyft-Bc/syft/frameworks/torch/pointers/pointer_tensor.py'

### Imports, paths

In [3]:

import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset


import syft as sy
from syft.frameworks.torch.federated import FederatedDataset, FederatedDataLoader, BaseDataset

print(torch.cuda.is_available())

W0726 09:43:26.033786 139958796494720 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0726 09:43:26.057780 139958796494720 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



True


In [0]:
# paths to training and test data
data_dir = '/content/gdrive/My Drive/Colab Notebooks/flower_data/'
train_dir = data_dir + 'valid' # 'train'
valid_dir = data_dir + 'test' #'valid'

#os.chdir("/content/gdrive/My Drive/Colab Notebooks/")
#test_dir = data_dir + 'test'

### Architecture and helpers

In [0]:
# Make data loader based on the selected pre-trained model
def get_datasets():
    print('returning datasets')
    # ResNet, DenseNet expect 224, Inception expects 299
    #img_size = 299 if base == 'Inception' else 224 
    img_size = 128

    transforms_train = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    transforms_test = transforms.Compose([
        transforms.Resize(img_size + 1),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    # Load the datasets with ImageFolder
    trainset = datasets.ImageFolder(train_dir, transform=transforms_train)
    validationset = datasets.ImageFolder(valid_dir, transform=transforms_train)
    testset = datasets.ImageFolder(valid_dir, transform=transforms_test)
       
    return trainset, validationset, testset

In [0]:
#transforms a torch.Dataset or a sy.BaseDataset into a sy.FederatedDataset. 
def get_federated_dataset(dataset, workers):
    print('get_federated_dataset …')
    
    datasets = []
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, drop_last=True)
    for dataset_idx, (data, targets) in enumerate(data_loader):
        worker = workers[dataset_idx % len(workers)]
        data = data.send(worker)
        targets = targets.send(worker)
        datasets.append(BaseDataset(data, targets))  # .send(worker)

    print("dataset_federate Done!")
    return FederatedDataset(datasets)

### Architecture

In [39]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(3*28*28, 1024)
        self.fc2 = nn.Linear(1024, 102)

    def forward(self, x):
        #print(x.shape)
        x = x.view(-1, 3*28*28)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
      
      
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 4, 2, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 4, 2, padding=1)
        self.fc1 = nn.Linear(32*8*8, 2048) # depth * height*width
        self.fc2 = nn.Linear(2048, 102)

# (in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
    
    def forward(self, x):
        #print('x {}, size {}'.format(x.shape, x.size(0))) # x torch.Size([32, 3, 128, 128]), size 0
        x = F.relu(self.conv1(x))
        #print('in {}'.format(x.shape))  # in torch.Size([32, 16, 64, 64])
        
        x = F.max_pool2d(x, 2, 2)
        #print('max_pool2d {}'.format(x.shape)) # max_pool2d torch.Size([32, 16, 32, 32])
        
        x = F.relu(self.conv2(x))
        #print('conv2 {}'.format(x.shape)) # conv2 torch.Size([32, 32, 16, 16])
        
        x = F.max_pool2d(x, 2, 2) 
        #print('max_pool2d {}'.format(x.shape)) # max_pool2d torch.Size([32, 32, 8, 8])
        
        x = x.view(-1, 32*8*8) # depth * height*width after maxPool
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
      








    
# shape '[-1, 140450]' is invalid for input of size 1345600

'''We can compute the spatial size of the output volume as a function of 
the input volume size (W), 
the kernel/filter size (F), 
the stride with which they are applied (S), 
and the amount of zero padding used (P) on the border. 
The correct formula for calculating how many neurons define the output_W 
is given by (W−F+2P)/S +1.

For example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 
we would get a 5x5 output. With stride 2 we would get a 3x3 output.'''

'We can compute the spatial size of the output volume as a function of \nthe input volume size (W), \nthe kernel/filter size (F), \nthe stride with which they are applied (S), \nand the amount of zero padding used (P) on the border. \nThe correct formula for calculating how many neurons define the output_W \nis given by (W−F+2P)/S +1.\n\nFor example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 \nwe would get a 5x5 output. With stride 2 we would get a 3x3 output.'

### Helpers, visualize

In [0]:
# Helper functions for printing oput training progress data
def print_epoch_start_stats(e_start, e_end, current_lr, current_vmin):

    print('*** Epoch [{}/{}]: Training with LR [{:.6f}], current VLoss Min [{:.4f}]'.format(
    e_start, e_end, current_lr, current_vmin))

def print_epoch_end_stats(train_loss, valid_loss, valid_acc, epoch_time):

    print('   Train loss: \t{:.6f}'.format(train_loss))
    print('   Valid loss: \t{:.6f}'.format(valid_loss))
    print('   Valid acc: \t{:.6f}'.format(valid_acc))
    print('*** Epoch completed in {:.0f}m {:.0f}s'.format(epoch_time // 60, epoch_time % 60))   
    
    
    
import datetime

def get_time():
      hour = datetime.datetime.today().hour +2
      minute = datetime.datetime.today().minute
      second = datetime.datetime.today().second
      return hour, minute, second


In [0]:
# Visualize plot
def plot_loss_acc(n_epochs, train_losses, valid_losses, valid_accuracies):
    fig, (ax1, ax2) = plt.subplots(figsize=(14,6), ncols=2)
    ax1.plot(valid_losses, label='Validation loss')
    ax1.plot(train_losses, label='Training loss')
    ax1.legend(frameon=False)
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    #x_ticks = [x for x in range(0,n_epochs,2)]
    #plt.xticks(x_ticks)
    
    ax2.plot(valid_accuracies, label = 'Validation accuracy')
    ax2.legend(frameon=False)
    ax2.set_xlabel('Epochs')
    
    plt.tight_layout()

### Helpers train, validate

In [0]:
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
       values taken from a normal distribution.'''
    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        n = m.in_features
        # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0, 1/np.sqrt(n))
        # m.bias.data should be 0
        m.bias.data.fill_(0)
        


def train_epoch(model, dataloader, criterion, optimizer, train_on_gpu=False):
    # initialize variables to monitor training and validation loss
    train_loss = 0.0
    train_accuracy = 0.0
    correct = 0.0
    total = 0.0
    
    for batch_idx, (data, target) in enumerate(dataloader):
        # move to GPU
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        model.send(data.location) # send the model to the right location
        
        ## find the loss and update the model parameters accordingly
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        model.get() # get the model back
        current_loss = loss.get()
        
        # get the loss per batch and accumulate
        train_loss += current_loss.item()
        
        # get the class, highest probability
        probabilities = torch.exp(output)
        _, top_class = probabilities.topk(1, dim=1)
        
        # check if the predicted class is correct
        equals = top_class == target.view(*top_class.shape)
        
        acc = torch.mean(torch.tensor(equals))
        train_accuracy += acc
    return train_loss , train_accuracy


def validate_epoch(model, dataloader, criterion, train_on_gpu=False):
    valid_loss = 0.0
    valid_accuracy = 0.0
    correct = 0.0
    total = 0.0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            # move to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output,target)
            
            valid_loss += loss.item()

            ps = torch.exp(output)
            _ , top_class = ps.topk(1,dim = 1)
            #_, top_class = torch.max(ps, dim=1)
            equals = top_class == target.view(*top_class.shape) # shape is (batch size x 1)
            valid_accuracy += torch.mean(equals.type(torch.FloatTensor))
            
            #pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            #correct += pred.eq(target.view_as(pred)).sum().item()

    return valid_loss, valid_accuracy

In [0]:
def train_my_model(n_epochs, loaders, model, optimizer, criterion, scheduler, use_cuda=False):
    print('Training started at ', get_time())
    
    valid_losses = []
    train_losses = []
    valid_accuracies = []
    
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(n_epochs):
        
         # initialize variables to monitor training and validation loss
        training_loss = 0.0
        training_accuracy = 0.0
    
        #if scheduler is not None:
          #scheduler.step()
        
        ###################
        # train the model #
        model.train()
        training_loss, training_accuracy = train_epoch(model, loaders[0], criterion, optimizer, use_cuda)
    
        
        ######################    
        # validate the model #
        model.eval()
        validation_loss, validation_accuracy = validate_epoch(model, loaders[1], criterion, use_cuda) #validation_accuracy
        
        #if scheduler is not None:
          #scheduler.step(validation_loss)
        
        ###### print training/validation statistics 
        # calculate the average loss per epoch
        training_loss = training_loss/len_trainloader
        train_losses.append(training_loss)
        #temp_n1 = loaders[0].get()
        #print('temp_n1 {}; shape: {}'.format(temp_n1, temp_n1.shape))
        training_accuracy = training_accuracy/len_trainloader

        validation_loss = validation_loss/len_validloader
        valid_losses.append(validation_loss)
        #temp_n = loaders[1]
        
        validation_accuracy = validation_accuracy/len_validloader
        valid_accuracies.append(validation_accuracy)
        
        hour, minute, second = get_time()
        print('Epoch: {} at {}:{}:{} \tTrain. Loss: {:.6f} \tValid. Loss: {:.6f} \t Accur.: {:.6f}'.format(
                  epoch,
                  hour, minute, second,
                  training_loss,
                  #training_accuracy, 
                  validation_loss,
                  validation_accuracy ))
        
        ###### TODO: save the model if validation loss has decreased
        if validation_loss <= valid_loss_min:
            print('Validation loss decreased by {:.6f}'.format(validation_loss - valid_loss_min))
            #torch.save(model.state_dict(), save_path)
            valid_loss_min = validation_loss
            
            
    ##### visualize
    plot_loss_acc(n_epochs, train_losses, valid_losses, valid_accuracies)
    
    return model

### Execute

In [0]:
# create workers, 
hook = sy.TorchHook(torch)

ada = sy.VirtualWorker(hook, 'ada')
bob = sy.VirtualWorker(hook, 'bob')
cyd = sy.VirtualWorker(hook, 'cyd')

In [31]:
# Create the data loaders, federated PySyft loader
datasets.ImageFolder.federate = get_federated_dataset

trainset, validset, _ = get_datasets()

fed_train_loader = sy.FederatedDataLoader(trainset.federate((ada, bob, cyd)),
                                          batch_size=32, shuffle=True)

valid_loader = torch.utils.data.DataLoader(validset, batch_size=32)
#test_loader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)
  

print(fed_train_loader.workers)

len_trainloader = len(fed_train_loader)
len_validloader = len(valid_loader)
print(len_trainloader)


returning datasets
get_federated_dataset …
dataset_federate Done!
['ada', 'bob', 'cyd']
3


In [14]:
# for testing
data, labels = next(iter(fed_train_loader))
print(data)

print(f'objects of ada= {len(ada._objects)}, bob= {len(bob._objects)}, cyd= {len(cyd._objects)}')


(Wrapper)>[PointerTensor | me:14221569588 -> ada:20768126956]
objects of ada= 4, bob= 2, cyd= 2


## Start the training

In [40]:
print(f'objects of ada= {len(ada._objects)}, bob= {len(bob._objects)}, cyd= {len(cyd._objects)}')
#print(device)
torch.manual_seed(1)

model = MyNet() # Model()
#model.apply(weights_init_normal)

n_epochs = 20
lr = 0.01

optimizer = optim.SGD(model.parameters(), lr=lr) # TODO momentum is not supported at the moment
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 4)
criterion = nn.NLLLoss()


#model.to(device)
print('')
loaders = [fed_train_loader, valid_loader]
train_my_model(n_epochs, loaders, model, optimizer, criterion, scheduler)



objects of ada= 96, bob= 6, cyd= 6

Training started at  (14, 31, 41)


/content/PySyft-Bc/syft/frameworks/torch/hook/hook.py:764: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


Epoch: 0 at 14:33:54 	Train. Loss: 4.622559 	Valid. Loss: 4.649259 	 Accur.: 0.000000
Validation loss decreased by -inf
Epoch: 1 at 14:33:59 	Train. Loss: 4.458641 	Valid. Loss: 4.674637 	 Accur.: 0.000000
Epoch: 2 at 14:34:4 	Train. Loss: 4.255629 	Valid. Loss: 4.722991 	 Accur.: 0.000000
Epoch: 3 at 14:34:9 	Train. Loss: 3.931659 	Valid. Loss: 4.866782 	 Accur.: 0.000000
Epoch: 4 at 14:34:14 	Train. Loss: 3.471017 	Valid. Loss: 5.260429 	 Accur.: 0.000000
Epoch: 5 at 14:34:19 	Train. Loss: 3.170437 	Valid. Loss: 5.633280 	 Accur.: 0.000000
Epoch: 6 at 14:34:23 	Train. Loss: 3.038600 	Valid. Loss: 5.853168 	 Accur.: 0.000000
Epoch: 7 at 14:34:28 	Train. Loss: 2.935291 	Valid. Loss: 6.049929 	 Accur.: 0.000000
Epoch: 8 at 14:34:33 	Train. Loss: 2.843316 	Valid. Loss: 6.169907 	 Accur.: 0.000000
Epoch: 9 at 14:34:38 	Train. Loss: 2.758924 	Valid. Loss: 6.296890 	 Accur.: 0.000000
Epoch: 10 at 14:34:43 	Train. Loss: 2.680470 	Valid. Loss: 6.421354 	 Accur.: 0.000000
Epoch: 11 at 14:34:48

KeyboardInterrupt: ignored

### Clear the worker

In [0]:
ada.clear_objects()
bob.clear_objects()
cyd.clear_objects()

<VirtualWorker id:cyd #objects:0>

In [0]:
'''for param in model.parameters():
    param.requires_grad = True
    
fc_in = model.classifier.in_features

transferclassifier = nn.Sequential(
                        nn.BatchNorm1d(fc_in),
                        nn.Linear(fc_in, 102)
                        )

#model.fc = transferclassifier # resnet
model.classifier = transferclassifier'''